In [23]:
from torchtext import data, datasets
import torch
import numpy  as np
from torch import nn, optim
import time
import torch.nn.functional as F

import spacy
nlp = spacy.load('en')

In [2]:
questions =  data.Field(tokenize = 'spacy', batch_first = True) # uses spacy for tokenization
labels = data.LabelField(dtype = torch.float) # sets as floats so we can caluclate gradients.

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [3]:
train_data, _ = datasets.TREC.splits(questions, labels)
train_data, valid_data = train_data.split()
print(len(train_data), len(valid_data))

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


3816 1636


In [4]:
# visualise sample
ith = np.random.randint(len(train_data))
print(train_data.examples[ith].text)
print(train_data.examples[ith].label)

['What', 'kind', 'of', 'rocket', 'launched', 'the', 'Surveyor', 'spacecraft', '?']
ENTY


In [5]:
questions.build_vocab(train_data, vectors="glove.6B.200d", unk_init=torch.Tensor.normal_)
labels.build_vocab(train_data)

In [6]:
questions.vocab.vectors

tensor([[-0.4890, -1.9077,  0.0959,  ...,  1.2058, -0.5414,  2.6896],
        [ 0.5859, -1.7986,  0.0973,  ...,  0.4793,  0.5992,  0.3551],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [ 0.2698, -0.3513,  0.0998,  ...,  0.1792,  0.0125, -0.0725],
        [ 0.1940, -0.4843, -0.7601,  ...,  0.3863, -0.6567, -0.0112],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570]])

In [7]:
# create data iterators
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = 64,
    device = device
)


/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [8]:
class CNN(nn.Module):
    def __init__(self,
                 vocab_size, 
                 embedding_dim, 
                 n_filters, 
                 filter_sizes, 
                 output_dim, 
                 dropout, 
                 pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size,
                                      embedding_dim=embedding_dim, 
                                      padding_idx=pad_idx) # padding_index ->index of embeddign used to pad to same length, define later manually
        self.convs = nn.ModuleList( # define a list of convs layers
            [nn.Conv2d(in_channels=1, # input text does not have channels
                       out_channels=n_filters, # how many conv filters with same shape to train (same for all filter_sizes) 
                       kernel_size=(fs, embedding_dim)
                      ) 
             for fs in filter_sizes]
        )
        
        self.fc = nn.Linear(len(filter_sizes)*n_filters, output_dim)
        self.dropout= nn.Dropout(dropout)
        
        
    def forward(self, text):
        emb = self.embedding(text).unsqueeze(1)
        conved = [F.relu(conv2d(emb)).squeeze(3) for conv2d in self.convs]
        pooled = [F.max_pool1d(c , c.shape[2]).squeeze(2) for c in conved]
        concat = self.dropout(torch.cat(pooled, dim=1))
        return self.fc(concat)
    

In [9]:
input_dimension = len(questions.vocab)
output_dimension = 6
embedding_dimension = 200
pad_index = questions.vocab.stoi[questions.pad_token]
number_of_filters = 100
filter_sizes = [2, 3, 4]
dropout_pc = 0.5

model = CNN(vocab_size=input_dimension, 
                 embedding_dim = embedding_dimension, 
                 n_filters = number_of_filters, 
                 filter_sizes = filter_sizes, 
                 output_dim = output_dimension, 
                 dropout=dropout_pc, 
                 pad_idx=pad_index)

In [10]:
# load glove as embeddings  for tokens
glove_embeddings = questions.vocab.vectors
model.embedding.weight.data.copy_(glove_embeddings)

tensor([[-0.4890, -1.9077,  0.0959,  ...,  1.2058, -0.5414,  2.6896],
        [ 0.5859, -1.7986,  0.0973,  ...,  0.4793,  0.5992,  0.3551],
        [ 0.3911,  0.4019, -0.1505,  ..., -0.0348,  0.0798,  0.5031],
        ...,
        [ 0.2698, -0.3513,  0.0998,  ...,  0.1792,  0.0125, -0.0725],
        [ 0.1940, -0.4843, -0.7601,  ...,  0.3863, -0.6567, -0.0112],
        [ 0.5741, -0.4343, -0.1119,  ...,  0.7629,  0.3831, -0.1570]])

In [11]:
# handels unknown and paddings
unknown_index = questions.vocab.stoi[questions.unk_token]
model.embedding.weight.data[unknown_index] = torch.zeros(embedding_dimension)
model.embedding.weight.data[pad_index] = torch.zeros(embedding_dimension)


In [12]:
optimizer = torch.optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss().to(device)
model = model.to(device)

In [13]:
# trainig the CNN
def multi_accuracy(preds, y):
    pred = torch.max(preds, 1).indices
    correct = (pred == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [14]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        preds = model(batch.text).squeeze(1)
        loss = criterion(preds, batch.label.long())
        acc = multi_accuracy(preds, batch.label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    total_epoch_loss = epoch_loss/len(iterator)
    total_epoch_accuracy = epoch_acc/len(iterator)
        
    return total_epoch_loss, total_epoch_accuracy

In [15]:
def model_eval(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            preds = model(batch.text).squeeze(1)
            loss = criterion(preds, batch.label.long())
            acc = multi_accuracy(preds, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    total_epoch_loss = epoch_loss/len(iterator)
    total_epoch_accuracy = epoch_acc/len(iterator)
        
    return total_epoch_loss, total_epoch_accuracy

In [16]:
epochs = 10
lowest_validation_loss = float('inf')

for epoch in range(epochs):
    start_time = time.time()
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = model_eval(model, valid_iterator, criterion)
    end_time = time.time()
    
    if valid_loss < lowest_validation_loss:
        lowest_validation_loss = valid_loss
        torch.save(model.state_dict(), 'cnn_model.pt')
        
    print(f"epoch {epoch}/{epochs}, time {int(end_time-start_time)}s")
    print(f"Train loss: {train_loss} train acc: {train_acc}")
    print(f"Valid loss: {valid_loss} Valid acc: {valid_acc}")

/Users/FrancescaSogaro/.pyenv/versions/3.8.2/lib/python3.8/site-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


epoch 0/10, time 4s
Train loss: 1.1837895661592484 train acc: 0.529583332935969
Valid loss: 0.9222323321379148 Valid acc: 0.6601228622289804
epoch 1/10, time 4s
Train loss: 0.7438710441191991 train acc: 0.7366666664679845
Valid loss: 0.7209972670445075 Valid acc: 0.7563434839248657
epoch 2/10, time 4s
Train loss: 0.4973691125710805 train acc: 0.8449999998013179
Valid loss: 0.6104290668780987 Valid acc: 0.7863915608479426
epoch 3/10, time 4s
Train loss: 0.34622849176327386 train acc: 0.8948437501986821
Valid loss: 0.5475277935083096 Valid acc: 0.8157051274409661
epoch 4/10, time 4s
Train loss: 0.22761213096479574 train acc: 0.9338541666666667
Valid loss: 0.5100480982890496 Valid acc: 0.8327991458085867
epoch 5/10, time 4s
Train loss: 0.16118539993961653 train acc: 0.9573958337306976
Valid loss: 0.4948240684775206 Valid acc: 0.8321981842701252
epoch 6/10, time 4s
Train loss: 0.10206299560765425 train acc: 0.9793749998013178
Valid loss: 0.48593093110964847 Valid acc: 0.8255876073470483
ep

In [17]:
# predictions

In [18]:
model.load_state_dict(torch.load("cnn_model.pt"))


<All keys matched successfully>

In [26]:
def predict_class(model, sentence, min_len=5):
    tokenized = [tok.text 
                 for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized = ['<pad>'] * (min_len - len(tokenized))
    indexed = [questions.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    
    model.eval()
    prediction = torch.max(model(tensor), 1).indices.item()
    pred_index = labels.vocab.itos[prediction]
    
    return pred_index
    

In [28]:
pred_class = predict_class(model, "How many road must a man walk down?")
pred_class

'NUM'